# Proyecto Final Analítica computacional para la toma de decisiones

Departamento de Ingeniería Industrial

Universidad de los Andes

Tecnológico de Monterrey
## Etapa 4b - Modelos predictivos Andes - Clasificación
1. Carga de librerías y configuración inicial
Importamos las librerías necesarias.

Configuramos la semilla para que los experimentos sean reproducibles.

In [2]:
import os
import random
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import tensorflow as tf
from tensorflow import keras

# --------------------------------------------------------------
# Fijamos semillas para reproducibilidad básica
# --------------------------------------------------------------
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)


2. Carga del conjunto de datos limpio
Cargamos el dataset limpio (listings_model_no_outliers.csv).

Verificamos las dimensiones y las primeras filas del dataframe.

In [3]:
# Cargar el dataset limpio sin outliers
df = pd.read_csv("C:/Users/Miguel A Ortiz/Documents/Analítica/Proyecto/2 SEGUNDA SUBIDA/listings_model_no_outliers.csv")

# Verificación inicial de las dimensiones
print(f"Dimensiones del dataset limpio: {df.shape}")
df.head()


Dimensiones del dataset limpio: (40550, 147)


,id,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,...,property_type_Shepherd’s hut,property_type_Tent,property_type_Tiny home,property_type_Tower,room_type_Hotel room,room_type_Private room,room_type_Shared room,host_is_superhost_No,host_is_superhost_Yes,instant_bookable_Yes
0,13913,51.56861,-0.11270,1,1.0,1.0,1.0,70.0,1,29,...,False,False,False,False,False,True,False,False,True,False
1,15400,51.48780,-0.16813,2,1.0,1.0,1.0,149.0,4,30,...,False,False,False,False,False,False,False,True,False,False
2,24328,51.47072,-0.16266,2,1.0,1.0,2.0,135.0,7,1125,...,False,False,False,False,False,False,False,True,False,False
3,36274,51.52322,-0.06979,4,1.0,1.0,0.0,210.0,5,1125,...,False,False,False,False,False,False,False,True,False,True
4,38995,51.47860,-0.06114,2,1.0,1.0,1.0,49.0,1,8,...,False,False,False,False,False,True,False,False,True,False


3. Preparación de datos para clasificación
Creamos las variables predictoras X y el objetivo y para la clasificación.

Imputamos los valores nulos en las características numéricas usando la mediana.

Realizamos un split de los datos en entrenamiento y prueba (train/test).

Normalizamos los datos utilizando StandardScaler.

In [4]:
print(df.columns)


Index(['id', 'latitude', 'longitude', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'price', 'minimum_nights', 'maximum_nights',
       ...
       'property_type_Shepherd’s hut', 'property_type_Tent',
       'property_type_Tiny home', 'property_type_Tower',
       'room_type_Hotel room', 'room_type_Private room',
       'room_type_Shared room', 'host_is_superhost_No',
       'host_is_superhost_Yes', 'instant_bookable_Yes'],
      dtype='object', length=147)


In [5]:
# Variables predictoras (X) y objetivo (y)
X = df.drop(columns=["price", "host_is_superhost_No", "host_is_superhost_Yes"])  # Eliminamos las columnas one-hot de 'host_is_superhost'
y = df["host_is_superhost_Yes"]  # Usamos 'host_is_superhost_Yes' como la variable objetivo de clasificación

# Imputar valores nulos en las características numéricas (usando la mediana)
X = X.fillna(X.median())

# División de los datos en train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=SEED
)

# Normalización de los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Datos preparados para clasificación.")


Datos preparados para clasificación.


4. Creación del modelo de clasificación (red neuronal)
Creamos una función para construir un modelo de red neuronal para clasificación binaria (superhost o no superhost).

Se utiliza sigmoid en la capa de salida para obtener probabilidades de clase.

binary_crossentropy se usa como función de pérdida, adecuada para clasificación binaria.

In [6]:
# Función para construir el modelo de clasificación con redes neuronales
def build_classification_model(input_dim, n_layers=2, n_units=64, learning_rate=1e-3):
    model = keras.Sequential(name="nn_classification_superhost")
    
    # Capa de entrada
    model.add(keras.layers.Input(shape=(input_dim,), name="input"))
    
    # Capas ocultas (activación ReLU)
    for i in range(n_layers):
        model.add(
            keras.layers.Dense(
                n_units,
                activation="relu",
                name=f"dense_hidden_{i+1}"
            )
        )
    
    # Capa de salida (activación sigmoide para clasificación binaria)
    model.add(
        keras.layers.Dense(
            1,
            activation="sigmoid",
            name="output_superhost"
        )
    )

    # Compilamos el modelo
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",  # Para clasificación binaria
        metrics=["accuracy"]
    )

    return model
